# Insurance Fraud Claim Prediction - Step 5: Model Deployment
Now that we have trained and selected our optimal model, its time to deploy it.  This notebook demonstrates how to user our Experiment and Pipelines from the previous steps to easly deploy our model as a Cortex Action. 

In [29]:
# Basic setup
%run config.ipynb

In [30]:
# Connect to Cortex 5 and create a Builder instance
cortex = Cortex.client()
builder = cortex.builder()

### Load the Experiement
Let's load our experiment from the previous step and find the model we want to deploy.

In [31]:
exp = cortex.experiment('claims-fraud/motorinsurancefraud-regression')
exp

---
The model created in the last run looks to be the best, let's deploy it

In [32]:
run = exp.get_run('4593yrn')
model = run.get_artifact('model')
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Model deployment - Step 1: Configure Data Pipeline for Inputs
Our model was trained with data that has had cleaning and feature engineering steps applied to it.  Since we want our users to send us the actual raw data, we need to deploy our pipeline to transform the input data into the form we expect.  This requires applying some of the same steps from before, but also requires us to remember some of the data created during model training such as the median values of certain columns and the final list of _dummy_ categorical columns created during feature engineering.  Luckily, our pipelines have a memory in the form of _context_ that we can reference here to achieve this.

In [33]:
train_ds = cortex.dataset('claims-fraud/motorinsurancefraud')

# Model our feature pipeline after the 'clean' pipeline
x_pipe = builder.pipeline('x_pipe')
x_pipe.from_pipeline(train_ds.pipeline('clean'))

In [34]:
# The dummy column conversion we did during training needs to be applied here.  Afterwards there will be missing columns because 
# our input instance will only contain at most one value per category.  We need to fill in the other expected columns.  We stored
# the expected set of columns in our pipeline so we can easily do this now.
def fix_columns(pipeline, df):
    all_cols = pipeline.get_context('columns')
    missing_cols = set(all_cols) - set(df.columns)
    for c in missing_cols:
        df[c] = 0
    
    # make sure we have all the columns we need
    assert(set(all_cols) - set(df.columns) == set())
    print (df.head())
    return df[all_cols]


In [35]:
# The feature engineering pipeline contains the complete list of dummy columns in addition to some steps we need
engineer_pipe = train_ds.pipeline('engineer')
x_pipe.set_context('columns', engineer_pipe.get_context('columns'))

# Reuse steps from our clean, features, and engineer pipelines
drop_unused = x_pipe.get_step('drop_unused')
fill_na_none = x_pipe.get_step('fill_na_none')
get_dummies = engineer_pipe.get_step('get_dummies')

# Build our final input pipeline
x_pipe.reset()
x_pipe.add_step(fill_na_none)
x_pipe.add_step(drop_unused)
x_pipe.add_step(get_dummies)
x_pipe.add_step(fix_columns)

### Model deployment - Step 2: Configure Data Pipeline for Output
If you remember, we scaled our target variable using the numpy _log1p_ function.  We need to inverse this using the _exp_ function so our predicted value is correct.

In [36]:
y_pipe = builder.pipeline('y_pipe')

In [37]:
def rescale_target(pipeline, df):
    df['Fraud Flag'] = np.exp(df['Fraud Flag'])

In [38]:
y_pipe.add_step(rescale_target)

### Model deployment - Step 3: Build and Deploy Cortex Action
Now that we have our input and output pipelines, we can use the Cortex Builder to package and deploy our model in one step.

In [39]:
builder.action('claims-fraud/motorinsurancefraud-predict')\
       .from_model(model, x_pipeline=x_pipe, y_pipeline=y_pipe, target='Fraud Flag')\
       .build()

Building Cortex Action (function): claims-fraud/motorinsurancefraud-predict
model version not found, pushing to remote storage: /cortex/models/claims-fraud/motorinsurancefraud-predict/2f278529ea82131fba9293e3d4974b7f.pk
Building Docker image private-registry.cortex-dev.insights.ai/skoorg53/claims-fraud_motorinsurancefraud-predict:9gi4y81...
Step 1/11 : FROM continuumio/miniconda3:4.5.4
Step 2/11 : WORKDIR /function
Step 3/11 : RUN apt-get update && apt-get install -y linux-headers-amd64 build-essential
Step 4/11 : RUN conda config --add channels conda-forge
Step 5/11 : COPY conda_requirements.txt .
Step 6/11 : RUN conda install --yes --file conda_requirements.txt
Step 7/11 : RUN pip install "dill==0.2.8.2" "fdk==0.0.31" "cortex-client==5.4a7"
Step 8/11 : COPY requirements.txt .
Step 9/11 : RUN pip install -r requirements.txt
Step 10/11 : COPY action.py .
Step 11/11 : ENTRYPOINT ["python", "action.py"]
Removing intermediate container f07485b7012b
Successfully built 46a0986d2be0
Successf

Name,Version,Type,Kind,Image,Deployment Status,claims-fraud/motorinsurancefraud-predict,12,function,python:3,private-registry.cortex-dev.insights.ai/skoorg53/claims-fraud_motorinsurancefraud-predict:9gi4y81,COMPLETED


In [40]:
action = cortex.action('claims-fraud/motorinsurancefraud-predict')
action

Name,Version,Type,Kind,Image,Deployment Status,claims-fraud/motorinsurancefraud-predict,12,function,python:3,private-registry.cortex-dev.insights.ai/skoorg53/claims-fraud_motorinsurancefraud-predict:9gi4y81,COMPLETED


---
Unit test for the Action.  Make sure our action is ready for use.

In [82]:
params = {
    "columns": ['ID','Insurance Type', 'Income of Policy Holder','Marital Status', 'Num Claimants', 'Injury Type', 'Overnight Hospital Stay', 'Claim Amount', 'Total Claimed', 'Num Claims', 'Num Soft Tissue', '% of Soft Tissue', 'Claim Amount Received'],
    "values": [[1,'CI',55000,'Married', 2,'Back','Yes',10000,15000,2,6,0.5,27000]]
}

result = action.invoke(message=cortex.message(params))
print(result.payload)
print()

{'columns': ['Fraud Flag'], 'values': [1]}


## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  We start by creating a Schema that defines our input for Ames Housing price prediction.  The schema will be built automatically using the parameters we already defined in our training dataset.

In [115]:
x_schema = builder.schema('claims-fraund/motorinsurancefraud').title('Motor Insurance Claims Fraud Test Instance').from_parameters(train_ds.parameters[0:][:-1]).build()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Ames Housing Price Prediction" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.

In [118]:
b = builder.skill('claims-fraud/motorinsurancefraud-predict').title('Motor Insurance Claims Fraud Prediction').description(' Predicts fraud for auto insurance claims')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action that we built previously to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [121]:
b = b.input('auto-claim').title('Auto Claim').use_schema(x_schema.name).all_routing(action, 'auto-claim-fraud-prediction').build()

In the previous step, we referenced an Output called **claim-prediction**.  We can create that Output here using the Output sub-builder.

In [124]:
b = b.output('auto-claim-fraud-prediction').title('Fraudulent Claim Prediction').parameter(name='Fraud Flag', type='number', format='double').build()

We can preview the CAMEL document our builder will create to make sure everything looks correct.

In [127]:
b.to_camel()

{'camel': '1.0.0',
 'name': 'claims-fraud/motorinsurancefraud-predict',
 'title': 'Motor Insurance Claims Fraud Prediction',
 'inputs': [{'name': 'auto-claim',
   'title': 'Auto Claim',
   'parameters': {'$ref': 'claims-fraund/motorinsurancefraud'},
   'routing': {'all': {'action': 'claims-fraud/motorinsurancefraud-predict',
     'output': 'auto-claim-fraud-prediction'}}}],
 'outputs': [{'name': 'auto-claim-fraud-prediction',
   'title': 'Fraudulent Claim Prediction',
   'parameters': [{'name': 'Fraud Flag',
     'type': 'number',
     'required': True,
     'format': 'double'}]}],
 'description': ' Predicts fraud for auto insurance claims'}

---
### Build and Publish the Skill to the Marketplace
This will build the Skill and publish it to my private marketplace.  It will then be available for use in the Agent Builder.

In [130]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

Motor Insurance Claims Fraud Prediction (claims-fraud/motorinsurancefraud-predict) v3
